In [1]:
import pandas as pd
import numpy as np
import pickle
import datetime as dt
import testdata_proc as pf
import global_variables as gv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [2]:
def calculateFuelcost(branch, vs, cat, nFast, chargers, pricing, pDict, meanprice):
    profile = pickle.load(open(
    "C:/Users/SofiaTaylor/OneDrive - Flexible Power Systems Ltd/dev/Smart_charging_prototypes/Outputs/LogsMixed/empty_profs{}{}{}.pkl".format(branch, vs, cat), 'rb')).reset_index()
    profile['from'] = pd.to_datetime(profile['from'])
    profile['date'] = pd.to_datetime(profile['date'])
    profile = profile.merge(pricing[['from', 'Tariff', 'Price']], on='from')
    profile['Charge'] = 0
    # profile = profile[profile['date'] == dt.datetime(2021, 3, 1)
    vehicles = profile['Vehicle_ID'].unique()

    energyReq = profile.groupby(['date', 'Vehicle_ID']).agg({'Battery_Use': 'sum'})
    energyReq['EnergyReq'] = energyReq['Battery_Use'] / 0.9
    for col in ['v', 'power1', 'remaining', 'FastCharger', 'ECostBAU']:
        energyReq[col] = 0
    for t in bands:
        energyReq['tp{}'.format(t)] = 0
        energyReq['deliv{}'.format(t)] = 0
    dates = profile['date'].unique()

    for date in dates:
        print(date)
        # print(date)
        dprof = profile[(profile['date'] == date) & (profile['Available'] == 1)]
        for v in vehicles:
            vprof = dprof[dprof['Vehicle_ID'] == v].copy()
            for t in bands:
                energyReq.loc[(date, v), 'tp{}'.format(t)] = len(vprof[vprof['Tariff'] == t])
            power = chargers[0] + energyReq.loc[(date, v), 'FastCharger'] * (chargers[1] - chargers[0])
            vprof['Charge'] = power/2
            vprof['CumCh'] = vprof['Charge'].cumsum() + energyReq.loc[(date, v), 'EnergyReq']
            vprof.loc[vprof['CumCh'] >= power/2, 'Charge'] = 0
            vprof.loc[(vprof['Charge'] > 0) & (vprof['CumCh'] >= 0), 'Charge'] = vprof['Charge'] - vprof['CumCh']
            vprof['CumCh'] = vprof['Charge'].cumsum() + energyReq.loc[(date, v), 'EnergyReq']
            vprof['ECost'] = vprof['Charge'] * vprof['Price']
            energyReq.loc[(date, v), 'ECostBAU'] = vprof['ECost'].sum() - vprof.iloc[-1].loc['CumCh']*meanprice

        energyReq['power1'] = -2*energyReq['EnergyReq'] / energyReq['tpGreenB']
        fastChargers = energyReq.loc[date].sort_values(by='power1', ascending=False).index.get_level_values('Vehicle_ID')[:nFast]
        for v in vehicles:
            power = chargers[0]
            req = -energyReq.loc[(date, v), 'EnergyReq']
            if v in fastChargers:
                energyReq.loc[(date, v), 'FastCharger'] = 1
                power = chargers[1]
            for t in bands:
                if 2*req/energyReq.loc[(date, v), 'tp{}'.format(t)] <= power:
                    energyReq.loc[(date, v), 'deliv{}'.format(t)] = req
                    req = 0
                else:
                    ch = power * energyReq.loc[(date, v), 'tp{}'.format(t)] / 2
                    energyReq.loc[(date, v), 'deliv{}'.format(t)] = ch
                    req += -ch

    energyReq['ECost'] = sum([energyReq['deliv{}'.format(t)] * pDict[t] for t in bands])
    energyReq.to_csv("C:/Users/SofiaTaylor/OneDrive - Flexible Power Systems Ltd/dev/Smart_charging_prototypes/Outputs/LogsMixed/SCApproc/enerqy_req{}{}{}.csv".format(branch, vs, chargers))

    print('Opt charge:', energyReq['ECost'].sum() + energyReq['remaining'].sum()*meanprice)
    print('BAU charge:', energyReq['ECostBAU'].sum())
    return

In [3]:
# Get pricing table
branch = 513
bands = ['GreenB', 'GreenP', 'AmberB', 'AmberP', 'RedP']
priceW = pd.read_excel('Inputs/Pricing2020_{}.xlsx'.format(branch), sheet_name=0, usecols=['Time', 'Tariff', 'Price'])
priceW['Time'] = pd.to_datetime(priceW['Time'], format= '%H:%M:%S' ).dt.time
priceWE = pd.read_excel('Inputs/Pricing2020_{}.xlsx'.format(branch), sheet_name=1, usecols=['Time', 'Tariff', 'Price'])
priceWE['Time'] = pd.to_datetime(priceWE['Time'], format= '%H:%M:%S' ).dt.time

timerange = pd.DataFrame(pd.date_range(
    start='1/1/2021 8:00', end='1/1/2022 7:30:00', freq='30min'), columns=['from'])
timerange['Weekday'] = timerange['from'].dt.weekday
timerange['Time'] = timerange['from'].dt.time
weekP = timerange.loc[timerange['Weekday'] < 5]
weekendP = timerange.loc[timerange['Weekday'] > 4]
pricing = pd.concat([
    weekP.merge(priceW, on='Time'),
    weekendP.merge(priceWE, on='Time')])
pricing['date'] = pricing['from'].dt.date
pricedf = pricing.groupby('Tariff').agg({'Price': 'mean'})
pDict = {t: pricedf.loc[t, 'Price'] for t in bands}
meanprice = pricing['Price'].mean()


In [8]:
# fig, ax = plt.subplots(
#     1,
#     figsize=(5,4),
#     gridspec_kw={'hspace':0.5})
# df1 = pricing[pricing['date'] == dt.date(2021, 1, 4)].sort_values(by='from')
# df2 = pricing[pricing['date'] == dt.date(2021, 1, 9)].sort_values(by='from')
# ax.plot(
#     df1['from'].dt.strftime('%H:%M'),
#     df1['Price']*100,
#     label = 'Weekday', color=gv.FPS_BLUE
# )
# ax.plot(
#     df2['from'].dt.strftime('%H:%M'),
#     df2['Price']*100,
#     label = 'Weekend', color=gv.FPS_GREEN
# )
# ax.set_xlabel('Time', color=gv.FPS_BLUE)
# ax.set_ylabel('p / kWh', color=gv.FPS_BLUE)
# ax.set_title(
#     'Electricity Price in Pontprennau',
#     color=gv.FPS_BLUE, fontweight='bold')
# ax.legend(frameon=False, bbox_to_anchor=(1, 1), loc='upper left')

# # majorFmt = mdates.DateFormatter('%H')
# ax.xaxis.set_major_locator(ticker.AutoLocator())
# fig.savefig(
#     'Outputs/OutputsJLP2/tariffs{}.png'.format(branch),
#     bbox_inches = "tight")
# fig.show()

In [9]:
# Get profile with pricing for an allocation
vs = ['Arrival111', 'Arrival111']
cat = '[22]'
nFast = 5
chargers = [7, 22]
calculateFuelcost(branch, vs, cat, nFast, chargers, pricing, pDict, meanprice)

2021-01-01T00:00:00.000000000
2021-01-02T00:00:00.000000000
2021-01-03T00:00:00.000000000
2021-01-04T00:00:00.000000000
2021-01-05T00:00:00.000000000
2021-01-06T00:00:00.000000000
2021-01-07T00:00:00.000000000
2021-01-08T00:00:00.000000000
2021-01-09T00:00:00.000000000
2021-01-10T00:00:00.000000000
2021-01-11T00:00:00.000000000
2021-01-12T00:00:00.000000000
2021-01-13T00:00:00.000000000
2021-01-14T00:00:00.000000000
2021-01-15T00:00:00.000000000
2021-01-16T00:00:00.000000000
2021-01-17T00:00:00.000000000
2021-01-18T00:00:00.000000000
2021-01-19T00:00:00.000000000
2021-01-20T00:00:00.000000000
2021-01-21T00:00:00.000000000
2021-01-22T00:00:00.000000000
2021-01-23T00:00:00.000000000
2021-01-24T00:00:00.000000000
2021-01-25T00:00:00.000000000
2021-01-26T00:00:00.000000000
2021-01-27T00:00:00.000000000
2021-01-28T00:00:00.000000000
2021-01-29T00:00:00.000000000
2021-01-30T00:00:00.000000000
2021-01-31T00:00:00.000000000
2021-02-01T00:00:00.000000000
2021-02-02T00:00:00.000000000
2021-02-03

In [6]:
# Get profile with pricing for an allocation
vs = ['Arrival111', 'Arrival111']
cat = '[22]'
nFast = 5
chargers = [11, 22]
calculateFuelcost(branch, vs, cat, nFast, chargers, pricing, pDict, meanprice)

2021-01-01T00:00:00.000000000
2021-01-02T00:00:00.000000000
2021-01-03T00:00:00.000000000
2021-01-04T00:00:00.000000000
2021-01-05T00:00:00.000000000
2021-01-06T00:00:00.000000000
2021-01-07T00:00:00.000000000
2021-01-08T00:00:00.000000000
2021-01-09T00:00:00.000000000
2021-01-10T00:00:00.000000000
2021-01-11T00:00:00.000000000
2021-01-12T00:00:00.000000000
2021-01-13T00:00:00.000000000
2021-01-14T00:00:00.000000000
2021-01-15T00:00:00.000000000
2021-01-16T00:00:00.000000000
2021-01-17T00:00:00.000000000
2021-01-18T00:00:00.000000000
2021-01-19T00:00:00.000000000
2021-01-20T00:00:00.000000000
2021-01-21T00:00:00.000000000
2021-01-22T00:00:00.000000000
2021-01-23T00:00:00.000000000
2021-01-24T00:00:00.000000000
2021-01-25T00:00:00.000000000
2021-01-26T00:00:00.000000000
2021-01-27T00:00:00.000000000
2021-01-28T00:00:00.000000000
2021-01-29T00:00:00.000000000
2021-01-30T00:00:00.000000000
2021-01-31T00:00:00.000000000
2021-02-01T00:00:00.000000000
2021-02-02T00:00:00.000000000
2021-02-03

In [7]:
# Get profile with pricing for an allocation
vs = ['Arrival111', 'Arrival111']
cat = '[22]'
nFast = 3
chargers = [7, 22]
calculateFuelcost(branch, vs, cat, nFast, chargers, pricing, pDict, meanprice)

2021-01-01T00:00:00.000000000
2021-01-02T00:00:00.000000000
2021-01-03T00:00:00.000000000
2021-01-04T00:00:00.000000000
2021-01-05T00:00:00.000000000
2021-01-06T00:00:00.000000000
2021-01-07T00:00:00.000000000
2021-01-08T00:00:00.000000000
2021-01-09T00:00:00.000000000
2021-01-10T00:00:00.000000000
2021-01-11T00:00:00.000000000
2021-01-12T00:00:00.000000000
2021-01-13T00:00:00.000000000
2021-01-14T00:00:00.000000000
2021-01-15T00:00:00.000000000
2021-01-16T00:00:00.000000000
2021-01-17T00:00:00.000000000
2021-01-18T00:00:00.000000000
2021-01-19T00:00:00.000000000
2021-01-20T00:00:00.000000000
2021-01-21T00:00:00.000000000
2021-01-22T00:00:00.000000000
2021-01-23T00:00:00.000000000
2021-01-24T00:00:00.000000000
2021-01-25T00:00:00.000000000
2021-01-26T00:00:00.000000000
2021-01-27T00:00:00.000000000
2021-01-28T00:00:00.000000000
2021-01-29T00:00:00.000000000
2021-01-30T00:00:00.000000000
2021-01-31T00:00:00.000000000
2021-02-01T00:00:00.000000000
2021-02-02T00:00:00.000000000
2021-02-03

In [10]:
# Get pricing table
branch = 457
bands = ['GreenB', 'GreenP', 'AmberB', 'AmberP', 'RedP']
priceW = pd.read_excel('Inputs/Pricing2020_{}.xlsx'.format(branch), sheet_name=0, usecols=['Time', 'Tariff', 'Price'])
priceW['Time'] = pd.to_datetime(priceW['Time'], format= '%H:%M:%S' ).dt.time
priceWE = pd.read_excel('Inputs/Pricing2020_{}.xlsx'.format(branch), sheet_name=1, usecols=['Time', 'Tariff', 'Price'])
priceWE['Time'] = pd.to_datetime(priceWE['Time'], format= '%H:%M:%S' ).dt.time

timerange = pd.DataFrame(pd.date_range(
    start='1/1/2021 8:00', end='1/1/2022 7:30:00', freq='30min'), columns=['from'])
timerange['Weekday'] = timerange['from'].dt.weekday
timerange['Time'] = timerange['from'].dt.time
weekP = timerange.loc[timerange['Weekday'] < 5]
weekendP = timerange.loc[timerange['Weekday'] > 4]
pricing = pd.concat([
    weekP.merge(priceW, on='Time'),
    weekendP.merge(priceWE, on='Time')])
pricing['date'] = pricing['from'].dt.date
pricedf = pricing.groupby('Tariff').agg({'Price': 'mean'})
pDict = {t: pricedf.loc[t, 'Price'] for t in bands}
meanprice = pricing['Price'].mean()


In [14]:
# Get profile with pricing for an allocation
vs = ['Arrival133', 'Arrival133']
cat = '[22, 60]'
nFast = 4
chargers = [7, 22]
calculateFuelcost(branch, vs, cat, nFast, chargers, pricing, pDict, meanprice)

2021-01-01T00:00:00.000000000
2021-01-02T00:00:00.000000000
2021-01-03T00:00:00.000000000
2021-01-04T00:00:00.000000000
2021-01-05T00:00:00.000000000
2021-01-06T00:00:00.000000000
2021-01-07T00:00:00.000000000
2021-01-08T00:00:00.000000000
2021-01-09T00:00:00.000000000
2021-01-10T00:00:00.000000000
2021-01-11T00:00:00.000000000
2021-01-12T00:00:00.000000000
2021-01-13T00:00:00.000000000
2021-01-14T00:00:00.000000000
2021-01-15T00:00:00.000000000
2021-01-16T00:00:00.000000000
2021-01-17T00:00:00.000000000
2021-01-18T00:00:00.000000000
2021-01-19T00:00:00.000000000
2021-01-20T00:00:00.000000000
2021-01-21T00:00:00.000000000
2021-01-22T00:00:00.000000000
2021-01-23T00:00:00.000000000
2021-01-24T00:00:00.000000000
2021-01-25T00:00:00.000000000
2021-01-26T00:00:00.000000000
2021-01-27T00:00:00.000000000
2021-01-28T00:00:00.000000000
2021-01-29T00:00:00.000000000
2021-01-30T00:00:00.000000000
2021-01-31T00:00:00.000000000
2021-02-01T00:00:00.000000000
2021-02-02T00:00:00.000000000
2021-02-03

In [121]:
# meanprice = 11.56 + 11.83 + 12.04 + 29.25
# branch = 513
# chargers = [7, 22]
# vs = ['Arrival89', 'Arrival111']
# nFast = 116
# profile = pickle.load(open(
#     "C:/Users/SofiaTaylor/OneDrive - Flexible Power Systems Ltd/dev/Smart_charging_prototypes/Outputs/LogsMixed/empty_profs{}{}{}.pkl".format(branch, vs, '[22]'), 'rb')).reset_index()
# profile['from'] = pd.to_datetime(profile['from'])
# profile['date'] = pd.to_datetime(profile['date'])
# profile['Time'] = profile['from'].dt.time
# profile['Tier'] = 0
# profile.loc[(profile['Time'] >= dt.time(21, 0, 0))
#             | (profile['Time'] < dt.time(7, 0, 0)), 'Tier'] = 1
# profile.loc[(profile['Time'] >= dt.time(19, 0, 0))
#             & (profile['Time'] < dt.time(21, 0, 0)), 'Tier'] = 2
# profile.loc[(profile['Time'] >= dt.time(7, 0, 0))
#             & (profile['Time'] < dt.time(16, 0, 0)), 'Tier'] = 3
# profile.loc[(profile['Time'] >= dt.time(16, 0, 0))
#             & (profile['Time'] < dt.time(19, 0, 0)), 'Tier'] = 4
# pDict = {1: 11.56, 2: 11.83, 3: 12.04, 4: 29.25}
# profile['p/kWh'] = profile['Tier'].map(pDict)
# profile['Charge'] = 0
# energyReq = profile.groupby(['date', 'Vehicle_ID']).agg({'Battery_Use': 'sum'})
# energyReq['EnergyReq'] = energyReq['Battery_Use'] / 0.9
# for col in ['v', 'tp1', 'tp2', 'tp3', 'tp4', 'power1', 'deliv1', 'deliv2', 'deliv3', 'deliv4', 'remaining', 'FastCharger', 'ECostBAU']:
#     energyReq[col] = 0
# dates = profile['date'].unique()
# fastChList = []
# for date in dates:
#     print(date)
#     dprof = profile[(profile['date'] == date) & (profile['Available'] == 1)]
#     for v in range(1, 117):
#         vprof = dprof[dprof['Vehicle_ID'] == v]
#         energyReq.loc[(date, v), 'tp1'] = len(vprof[vprof['Tier'] == 1])
#         energyReq.loc[(date, v), 'tp2'] = len(vprof[vprof['Tier'] == 2])
#         energyReq.loc[(date, v), 'tp3'] = len(vprof[vprof['Tier'] == 3])
#         energyReq.loc[(date, v), 'tp4'] = len(vprof[vprof['Tier'] == 4])
#     energyReq['power1'] = -2*energyReq['EnergyReq'] / energyReq['tp1']
#     fastChargers = energyReq.loc[date].sort_values(by='power1', ascending=False).index.get_level_values('Vehicle_ID')[:nFast]
#     fastChList.append([date, fastChargers])
#     for v in range(1, 117):
#         power = chargers[0]
#         req = -energyReq.loc[(date, v), 'EnergyReq']
#         if v in fastChargers:
#             energyReq.loc[(date, v), 'FastCharger'] = 1
#             power = chargers[1]
#         if energyReq.loc[(date, v), 'power1'] <= power:
#             energyReq.loc[(date, v), 'deliv1'] = req
#         else:
#             ch1 = power * energyReq.loc[(date, v), 'tp1'] / 2
#             energyReq.loc[(date, v), 'deliv1'] = ch1
#             req += -ch1
#             if 2*req/energyReq.loc[(date, v), 'tp2'] <= power:
#                 energyReq.loc[(date, v), 'deliv2'] = req
#             else:
#                 ch2 = power * energyReq.loc[(date, v), 'tp2'] / 2
#                 energyReq.loc[(date, v), 'deliv2'] = ch2
#                 req += -ch2
#                 if 2*req/energyReq.loc[(date, v), 'tp3'] <= power:
#                     energyReq.loc[(date, v), 'deliv3'] = req
#                 else:
#                     ch3 = power * energyReq.loc[(date, v), 'tp3'] / 2
#                     energyReq.loc[(date, v), 'deliv3'] = ch3
#                     req += -ch3
#                     if 2*req/energyReq.loc[(date, v), 'tp4'] <= power:
#                         energyReq.loc[(date, v), 'deliv4'] = req
#                     else:
#                         ch4 = power * energyReq.loc[(date, v), 'tp4'] / 2
#                         energyReq.loc[(date, v), 'deliv4'] = ch4
#                         energyReq.loc[(date, v), 'remaining'] = req - ch4
# energyReq['ECost'] = energyReq['deliv1'] * 11.56 + energyReq['deliv2'] * 11.83 + energyReq['deliv3'] * 12.04 + energyReq['deliv4'] * 29.25

# for date in dates:
#     dprof = profile[(profile['date'] == date) & (profile['Available'] == 1)]
#     print(date)
#     for v in range(1, 117):
#         vprof = dprof[dprof['Vehicle_ID'] == v]
#         power = chargers[0] + energyReq.loc[(date, v), 'FastCharger'] * (chargers[1] - chargers[0])
#         vprof['Charge'] = power/2
#         vprof['CumCh'] = vprof['Charge'].cumsum() + energyReq.loc[(date, v), 'EnergyReq']
#         vprof.loc[vprof['CumCh'] >= power/2, 'Charge'] = 0
#         vprof.loc[(vprof['Charge'] > 0) & (vprof['CumCh'] >= 0), 'Charge'] = vprof['Charge'] - vprof['CumCh']
#         vprof['CumCh'] = vprof['Charge'].cumsum() + energyReq.loc[(date, v), 'EnergyReq']
#         vprof['ECost'] = vprof['Charge'] * vprof['p/kWh']
#         energyReq.loc[(date, v), 'ECostBAU'] = vprof['ECost'].sum() - vprof.iloc[-1].loc['CumCh']*meanprice
# energyReq.to_csv("C:/Users/SofiaTaylor/OneDrive - Flexible Power Systems Ltd/dev/Smart_charging_prototypes/Outputs/LogsMixed/SCApproc/enerqy_req{}{}{}.csv".format(branch, vs, chargers))

# print('Opt charge:', energyReq['ECost'].sum() + energyReq['remaining'].sum()*meanprice)
# print('BAU charge:', energyReq['ECostBAU'].sum())

T00:00:00.000000000
2021-03-06T00:00:00.000000000
2021-03-07T00:00:00.000000000
2021-03-08T00:00:00.000000000
2021-03-09T00:00:00.000000000
2021-03-10T00:00:00.000000000
2021-03-11T00:00:00.000000000
2021-03-12T00:00:00.000000000
2021-03-13T00:00:00.000000000
2021-03-14T00:00:00.000000000
2021-03-15T00:00:00.000000000
2021-03-16T00:00:00.000000000
2021-03-17T00:00:00.000000000
2021-03-18T00:00:00.000000000
2021-03-19T00:00:00.000000000
2021-03-20T00:00:00.000000000
2021-03-21T00:00:00.000000000
2021-03-22T00:00:00.000000000
2021-03-23T00:00:00.000000000
2021-03-24T00:00:00.000000000
2021-03-25T00:00:00.000000000
2021-03-26T00:00:00.000000000
2021-03-27T00:00:00.000000000
2021-03-28T00:00:00.000000000
2021-03-29T00:00:00.000000000
2021-03-30T00:00:00.000000000
2021-03-31T00:00:00.000000000
2021-04-01T00:00:00.000000000
2021-04-02T00:00:00.000000000
2021-04-03T00:00:00.000000000
2021-04-04T00:00:00.000000000
2021-04-05T00:00:00.000000000
2021-04-06T00:00:00.000000000
2021-04-07T00:00:00.

Battery_Use  EnergyReq  v  tp1  tp2  tp3  tp4  \
date       Vehicle_ID                                                  
2021-01-01 1.0          -36.621040 -40.690045  0   16    4    2    6   
           2.0          -51.171063 -56.856736  0   17    2    0    0   
           3.0          -39.207998 -43.564442  0   17    4    2    6   
           4.0          -60.825831 -67.584257  0   13    0    0    0   
           5.0          -63.375443 -70.417159  0   13    0    0    0   
...                            ...        ... ..  ...  ...  ...  ...   
2021-12-31 112.0          0.000000   0.000000  0   20    4   18    6   
           113.0          0.000000   0.000000  0   20    4   18    6   
           114.0          0.000000   0.000000  0   20    4   18    6   
           115.0          0.000000   0.000000  0   20    4   18    6   
           116.0          0.000000   0.000000  0   20    4   18    6   

                          power1     deliv1  deliv2  deliv3  deliv4  \
date       Vehicle_ID                                                 
2021-01-01 1.0          5.086256  40.690045       0       0       0   
           2.0          6.689028  56.856736       0       0       0   
           3.0          5.125229  43.564442       0       0       0   
           4.0         10.397578  67.584257       0       0       0   
           5.0         10.833409  70.417159       0       0       0   
...                          ...        ...     ...     ...     ...   
2021-12-31 112.0       -0.000000  -0.000000       0       0       0   
           113.0       -0.000000  -0.000000       0       0       0   
           114.0       -0.000000  -0.000000       0       0       0   
           115.0       -0.000000  -0.000000       0       0       0   
           116.0       -0.000000  -0.000000       0       0       0   

                       remaining  FastCharger    ECostBAU       ECost  
date       Vehicle_ID                                                  
2021-01-01 1.0                 0            1  811.563804  470.376915  
           2.0                 0            1  663.203872  657.263872  
           3.0                 0            1  895.639937  503.604953  
           4.0                 0            1  781.274009  781.274009  
           5.0                 0            1  814.022352  814.022352  
...                          ...          ...         ...         ...  
2021-12-31 112.0               0            1    0.000000    0.000000  
           113.0               0            1    0.000000    0.000000  
           114.0               0            1    0.000000    0.000000  
           115.0               0            1    0.000000    0.000000  
           116.0               0            1    0.000000    0.000000  

[42340 rows x 16 columns]